# AVM Analysis




1. shopping_mall ↔ total_spend <br>
Which shopping mall generates the highest revenue?

2. shopping_mall ↔ avg_basket_value <br>
Does the average basket value differ across shopping malls?

3. shopping_mall ↔ transaction_count <br>
Traffic volume by shopping mall

4. shopping_mall ↔ season <br>
Seasonal variation in shopping mall performance

5. shopping_mall ↔ day‑of‑week <br>
Daily traffic patterns across shopping malls

6. shopping_mall ↔ Econ_Conf_level <br>
Which shopping mall is more resilient when consumer confidence declines?


7. shopping_mall ↔ category <br>
Which product categories are preferred at which shopping malls?